# 1. Configuration

In [1]:
# Import Libraries
import numpy as np
import pandas as pd
import scipy.optimize as opt
from sklearn.covariance import EmpiricalCovariance
from sklearn.linear_model import LinearRegression
import statistics
import pypfopt as pyp
from helper import *

# Adjust for Risk-free rate in columns of F.F 48 Industries Raw Dataset
df = pd.read_csv('Raw/gp_data_1986_to_2015.csv')
df1 = df.copy()
Rf = df['RF'].values.copy()

for column in df1.columns:
    if column in ['Month','Mkt-RF', 'RF']:
        continue
    df1[column] -= Rf
    
df1.columns = df1.columns.str.replace(' ', '')  
df2 = df1.iloc[:,3:]
 
# Setting the range for Backtesting
train = df1.iloc[:300,3:]         # 1986 to 2010
test = df1.iloc[300:,3:]          # 2010 to 2015

train_1 = df1.iloc[168:300,3:]    # 2000 Jan to 2010 Dec
val_1 = df1.iloc[300:,3:]         # 2011 Jan to 2015 Dec
train_2 = df1.iloc[168:295,3:]    # 2000 Jan to 2010 Jul
val_2 = df1.iloc[295:,3:]         # 2010 August to 2015 Dec
train_3 = df1.iloc[168:305,3:]    # 2000 Jan to 2011 May
val_3 = df1.iloc[305:,3:]         # 2011 June to 2015 Dec


# 2. Portfolios

### A. Baseline EWP and Market Portfolios

In [2]:
# EWP and evaluate its performance
EWP = ewp(train.shape[1])
EWP_perf = (evaluate_portfolio_performance_on_data(EWP, val_1)['Sharpe'] + 
            evaluate_portfolio_performance_on_data(EWP, val_2)['Sharpe'] +
            evaluate_portfolio_performance_on_data(EWP, val_3)['Sharpe'] )/3
print('Sharpe Ratio of EWP is', EWP_perf)  

# Market Portfolio
Market_perf = ( (df1.iloc[300:,:]['Mkt-RF'].mean())/(df1.iloc[300:,:]['Mkt-RF'].std()) + # Val 1
               (df1.iloc[295:,:]['Mkt-RF'].mean())/(df1.iloc[295:,:]['Mkt-RF'].std()) + # Val 2
               (df1.iloc[305:,:]['Mkt-RF'].mean())/(df1.iloc[305:,:]['Mkt-RF'].std()) # Val 3
              )/3
print("Sharpe ratio of Market Portfolio is", Market_perf)


Sharpe Ratio of EWP is 0.22229931983372567
Sharpe ratio of Market Portfolio is 0.29178255023325655


### B. Optimal Shrinkage Estimators

In [3]:
# Iterations of covariance shrinkage values in steps of 0.01 
cov_shrinkage = np.arange(0,1.1,0.1).tolist()
resultsshrinkage = pd.DataFrame([i for i in cov_shrinkage], 
                                columns=['Covariance Shrinkage'])

# Applying Covariance Shrinkage
resultsshrinkage['Portfolio Performance 1'] = resultsshrinkage['Covariance Shrinkage'].apply(lambda row: 
                                                        evaluate_portfolio_performance_on_data(
                                                                tangency(estimate_mu(train_1),shrinkcovariance(train_1, row)),
                                                                val_1))
resultsshrinkage['Portfolio Performance 2'] = resultsshrinkage['Covariance Shrinkage'].apply(lambda row: 
                                                        evaluate_portfolio_performance_on_data(
                                                                tangency(estimate_mu(train_2),shrinkcovariance(train_2, row)),
                                                                val_2))
resultsshrinkage['Portfolio Performance 3'] = resultsshrinkage['Covariance Shrinkage'].apply(lambda row: 
                                                        evaluate_portfolio_performance_on_data(
                                                                tangency(estimate_mu(train_3),shrinkcovariance(train_3, row)),
                                                                val_3))
# Finding Optimal Sharpe Ratio
Average_Sharpe_row = []
for i,j,k in zip(resultsshrinkage['Portfolio Performance 1'],
                 resultsshrinkage['Portfolio Performance 1'],
                 resultsshrinkage['Portfolio Performance 1']):
    Average_Sharpe_row.append(statistics.mean([i['Sharpe'], j['Sharpe'], j['Sharpe']]))
    
Index_Optimal = Average_Sharpe_row.index(max(Average_Sharpe_row))

print(f"Optimal Covariance Shrinkage is {resultsshrinkage.loc[Index_Optimal]['Covariance Shrinkage']} with an average sharpe ratio of {max(Average_Sharpe_row)}")


Optimal Covariance Shrinkage is 0.1 with an average sharpe ratio of 0.040485490434895396


### C. Black Litterman

In [4]:
# BL model (Idzorek's method) as per https://www.cis.upenn.edu/~mkearns/finread/idzorek.pdf

# Input A. Covariance Matrix
cov = df2.cov()  

# Input B: Implied Equilibrium Returns (Computed on a Monthly Basis based on Df2)
asset_weights = df2.sum()/(df2.sum().sum())  # Proxied as a weight of overall grand returns of 48 industries
global_return = df2.mean().multiply(asset_weights.values).sum()
market_var = np.matmul(asset_weights.values.reshape(len(asset_weights)).T,
                       np.matmul(cov.values, asset_weights.values.reshape(len(asset_weights))))
risk_aversion = global_return / market_var
implied_equilibrium_returns = implied_rets(risk_aversion, cov, asset_weights) 

# Input C: Defining Absolute Views for each industries
view = pd.DataFrame({'Industries':df2.columns, 'View':[0]*48, 'Confidence':[0]*48})
sector1 = [x.strip() for x in 'Agric, Food, Smoke, Soda, Beer, Toys, Fun, Books, Hshld, Clths, Paper, Util, Oil, Coal, Mines, Gold'.split(',')]
sector2 = [x.strip() for x in 'Hlth, MedEq, Drugs'.split(',')]
sector3 = [x.strip() for x in 'Fin, RlEst, Insur, Banks'.split(',')]
sector4 = [x.strip() for x in 'Meals, Rtail, Whlsl, BusSv, PerSv'.split(',')]
sector5 = [x.strip() for x in 'Trans, Boxes, LabEq, Ships, Aero, Autos, ElcEq, Mach, FabPr, Steel, Cnstr, BldMt, Txtls, Rubbr, Chems, Other, Guns '.split(',')]
sector6 = [x.strip() for x in 'Chips, Comps, Telcm'.split(',')]
view.loc[view['Industries'].isin(sector1), 'View'] = 2/12 # Commodities
view.loc[view['Industries'].isin(sector2), 'View'] = 15/12 # Healthcare
view.loc[view['Industries'].isin(sector3), 'View'] = 12/12 # Fin services
view.loc[view['Industries'].isin(sector4), 'View'] = 2/12 # Services
view.loc[view['Industries'].isin(sector5), 'View'] = 3/12 # Industrial
view.loc[view['Industries'].isin(sector6), 'View'] = 15/12 # Tech

# Input D: Confidence of views for each industry using Idzorek's Model 
view.loc[view['Industries'].isin(sector1), 'Confidence'] = 0.30
view.loc[view['Industries'].isin(sector2), 'Confidence'] = 0.65
view.loc[view['Industries'].isin(sector3), 'Confidence'] = 0.60
view.loc[view['Industries'].isin(sector4), 'Confidence'] = 0.30
view.loc[view['Industries'].isin(sector5), 'Confidence'] = 0.30
view.loc[view['Industries'].isin(sector6), 'Confidence'] = 0.65

# Black Litterman Idzorek's method 
view = view.set_index('Industries')
views_dict = {ind : view['View'][ind] for ind in view.index}
confidences = list(view.Confidence)

# Train our Black Litterman model
bl_confi = pyp.BlackLittermanModel(cov,                            # Input A
                                   pi=implied_equilibrium_returns, # Input B
                                   absolute_views=views_dict,      # Input C
                                   omega="idzorek",                # Use Idzorek BL Model 
                                   view_confidences=confidences)   # Input D
bl_confi_weights = bl_confi.bl_weights() 

In [5]:
# Black Litterman Model Comparison of Historical, Implied and BL returns
bl_confi_weights = bl_confi.bl_weights() 
bl_comparision_returns = pd.DataFrame([implied_equilibrium_returns, 
                                       df2.mean(), 
                                       pd.Series(views_dict), 
                                       bl_confi.bl_returns()], 
                                     index=['Implied Returns', 'Historical Returns','Views', 'Black Litterman Returns']).T

bl_comparision_returns = bl_comparision_returns.reset_index(level=0)
bl_comparision_returns.columns = ['Industry','Implied Returns', 'Historical Returns','Views', 'Black Litterman Returns']
bl_comparision_returns['Sector'] = ['0']*48
bl_comparision_returns.loc[bl_comparision_returns['Industry'].isin(sector1), 'Sector'] = 'Sector 1: Bearish'
bl_comparision_returns.loc[bl_comparision_returns['Industry'].isin(sector2), 'Sector'] = 'Sector 2: Bullish'
bl_comparision_returns.loc[bl_comparision_returns['Industry'].isin(sector3), 'Sector'] = 'Sector 3: Bullish'
bl_comparision_returns.loc[bl_comparision_returns['Industry'].isin(sector4), 'Sector'] = 'Sector 4: Bearish'
bl_comparision_returns.loc[bl_comparision_returns['Industry'].isin(sector5), 'Sector'] = 'Sector 5: Bearish'
bl_comparision_returns.loc[bl_comparision_returns['Industry'].isin(sector6), 'Sector'] = 'Sector 6: Bullish'

bl_comparision_returns = bl_comparision_returns[['Sector', 'Industry','Historical Returns','Implied Returns','Views','Black Litterman Returns' ]]
bl_comparision_returns = bl_comparision_returns.set_index('Sector')
bl_comparision_returns = bl_comparision_returns.loc[['Sector 1: Bearish','Sector 2: Bullish','Sector 3: Bullish',
                                                     'Sector 4: Bearish','Sector 5: Bearish','Sector 6: Bullish']]
bl_comparision_returns


,Industry,Historical Returns,Implied Returns,Views,Black Litterman Returns
Sector,,,,,
Sector 1: Bearish,Agric,0.700861,0.611881,0.166667,0.372015
Sector 1: Bearish,Food,0.836806,0.470870,0.166667,0.446795
Sector 1: Bearish,Soda,0.899500,0.692122,0.166667,0.479937
Sector 1: Bearish,Beer,0.991889,0.496143,0.166667,0.454350
Sector 1: Bearish,Smoke,1.254278,0.533675,0.166667,0.446339
Sector 1: Bearish,Toys,0.531028,0.840393,0.166667,0.451464
Sector 1: Bearish,Fun,0.940722,1.029482,0.166667,0.766135
Sector 1: Bearish,Books,0.480139,0.772544,0.166667,0.627902
Sector 1: Bearish,Hshld,0.667972,0.542693,0.166667,0.431022


In [6]:
# Black Litterman Model Comparison of BL weights vs Market Capitalisation weights
bl_comparision_weights =pd.DataFrame(list(zip([i for i in bl_confi_weights],
              [i for i in bl_confi.bl_weights().values()],
              [i for i in asset_weights])),
             columns=['Industry','Black Litterman Weights','Market Capitalization Weights'])
bl_comparision_weights['Sector'] = ['0']*48
bl_comparision_weights.loc[bl_comparision_weights['Industry'].isin(sector1), 'Sector'] = 'Sector 1: Bearish'
bl_comparision_weights.loc[bl_comparision_weights['Industry'].isin(sector2), 'Sector'] = 'Sector 2: Bullish'
bl_comparision_weights.loc[bl_comparision_weights['Industry'].isin(sector3), 'Sector'] = 'Sector 3: Bullish'
bl_comparision_weights.loc[bl_comparision_weights['Industry'].isin(sector4), 'Sector'] = 'Sector 4: Bearish'
bl_comparision_weights.loc[bl_comparision_weights['Industry'].isin(sector5), 'Sector'] = 'Sector 5: Bearish'
bl_comparision_weights.loc[bl_comparision_weights['Industry'].isin(sector6), 'Sector'] = 'Sector 6: Bullish'

bl_comparision_weights = bl_comparision_weights[['Sector', 'Industry','Black Litterman Weights','Market Capitalization Weights']]
bl_comparision_weights = bl_comparision_weights.set_index('Sector')
bl_comparision_weights = bl_comparision_weights.loc[['Sector 1: Bearish','Sector 2: Bullish','Sector 3: Bullish',
                                                     'Sector 4: Bearish','Sector 5: Bearish','Sector 6: Bullish']]
bl_comparision_weights['Black Litterman Weights greater than Market Cap Weights?'] = np.where(bl_comparision_weights['Black Litterman Weights'] > bl_comparision_weights['Market Capitalization Weights'],
                                                                                              'True (Overweight)', 'False (Underweight)')
bl_comparision_weights

,Industry,Black Litterman Weights,Market Capitalization Weights,Black Litterman Weights greater than Market Cap Weights?
Sector,,,,
Sector 1: Bearish,Agric,-0.034896,0.020862,False (Underweight)
Sector 1: Bearish,Food,-0.127521,0.024909,False (Underweight)
Sector 1: Bearish,Soda,-0.044315,0.026775,False (Underweight)
Sector 1: Bearish,Beer,-0.090870,0.029526,False (Underweight)
Sector 1: Bearish,Smoke,-0.030741,0.037336,False (Underweight)
Sector 1: Bearish,Toys,-0.053151,0.015807,False (Underweight)
Sector 1: Bearish,Fun,-0.085419,0.028002,False (Underweight)
Sector 1: Bearish,Books,-0.136018,0.014292,False (Underweight)
Sector 1: Bearish,Hshld,-0.120562,0.019883,False (Underweight)


In [7]:
# Black Litterman with no shrinkage applied to Covariance Matrix
print('Black Litterman Returns:',bl_confi.portfolio_performance()[0]) 
print('Black Litterman Risk:',bl_confi.portfolio_performance()[1]) 
print('Black Litterman Sharpe ratio:',bl_confi.portfolio_performance()[2]) 


Black Litterman Returns: 1.7029153814591462
Black Litterman Risk: 6.530099723595279
Black Litterman Sharpe ratio: 0.2577166433428651


# 3. Chosen Portfolio - Black Litterman Constrained

In [8]:
# Output - make Black Litterman realistic since there's heavy shorts and long positions
# No shorting and long constraints of no more than 10%
ef = pyp.EfficientFrontier(bl_confi.bl_returns(), 
                           bl_confi.bl_cov(), 
                           weight_bounds=(0,0.1)) # No short sale constraint; long position capped at 10%
Optimal_Weights_BL = pd.DataFrame(ef.max_sharpe().items(), columns = ['Industries', 'Optimal Weights'])
Optimal_Weights_BL

,Industries,Optimal Weights
0,Agric,0.000000
1,Food,0.077754
2,Soda,0.000000
3,Beer,0.000000
4,Smoke,0.000000
5,Toys,0.000000
6,Fun,0.000000
7,Books,0.000000
8,Hshld,0.000000
9,Clths,0.000000


# 4. Portfolio Stress Testing (VaR & ES)

In [9]:
# A. Market Portfolio Stress Testing on Validation DataSet - MKT Portfolio Column 
val_1_mkt = df1.iloc[300:,:2]
val_1_mkt.reset_index(inplace = True)
val_2_mkt = df1.iloc[295:,:2]
val_2_mkt.reset_index(inplace = True)
val_3_mkt = df1.iloc[305:,:2]
val_3_mkt.reset_index(inplace = True)
val_1_mkt_stress_test = stress_test_mkt(val_df=val_1_mkt, alpha=0.9)
val_2_mkt_stress_test = stress_test_mkt(val_df=val_2_mkt, alpha=0.9)
val_3_mkt_stress_test = stress_test_mkt(val_df=val_3_mkt, alpha=0.9)

print('The average VaR for the market portfolio is ' 
      + str(np.mean([val_1_mkt_stress_test['VaR'], val_2_mkt_stress_test['VaR'], val_3_mkt_stress_test['VaR']])))
print('The average ES for the market portfolio is ' 
      + str(np.mean([val_1_mkt_stress_test['ES'], val_2_mkt_stress_test['ES'], val_3_mkt_stress_test['ES']])))
print('\n')

# B. EWP Portfolio Stress Testing on Validation DataSet
w_e = ewp(df2.shape[1])
val_1_ewp_stress_test = stress_test(val_1, w_e, alpha=0.9)
val_2_ewp_stress_test = stress_test(val_2, w_e, alpha=0.9)
val_3_ewp_stress_test = stress_test(val_3, w_e, alpha=0.9)

print('The average VaR for the ewp portfolio is ' 
      + str(np.mean([val_1_ewp_stress_test['VaR'], val_2_ewp_stress_test['VaR'], val_3_ewp_stress_test['VaR']])))
print('The average ES for the ewp portfolio is ' 
      + str(np.mean([val_1_ewp_stress_test['ES'], val_2_ewp_stress_test['ES'], val_3_ewp_stress_test['ES']])))
print('\n')

# C. Optimal Shrinkage on Tangency Portfolio Stress Testing on Validation Dataset
optimal_1 = tangency(estimate_mu(train_1),
                     shrinkcovariance(train_1, 0.14))
optimal_2 = tangency(estimate_mu(train_2),
                     shrinkcovariance(train_2, 0.14))
optimal_3 = tangency(estimate_mu(train_3),
                     shrinkcovariance(train_3, 0.14))
optimal_weights = (optimal_1+optimal_2+optimal_3)/3

opt_1_chosen_stress_test = stress_test(val_1, optimal_weights, alpha=0.9)
opt_2_chosen_stress_test = stress_test(val_2, optimal_weights, alpha=0.9)
opt_3_chosen_stress_test = stress_test(val_3, optimal_weights, alpha=0.9)

print('The average VaR for the opt portfolio is ' 
      + str(np.mean([opt_1_chosen_stress_test['VaR'], opt_2_chosen_stress_test['VaR'], opt_3_chosen_stress_test['VaR']])))
print('The average ES for the opt portfolio is ' 
      + str(np.mean([opt_1_chosen_stress_test['ES'], opt_2_chosen_stress_test['ES'], opt_3_chosen_stress_test['ES']])))
print('\n')

# D. Black Litterman (Unconstrained Weights) Stress Testing on Validation Dataset
bl_confi_weights_array = np.array(list(bl_confi.bl_weights().values()))
bl1_stress_test = stress_test(val_1, bl_confi_weights_array, alpha=0.9)
bl2_stress_test = stress_test(val_2, bl_confi_weights_array, alpha=0.9)
bl3_stress_test = stress_test(val_3, bl_confi_weights_array, alpha=0.9)

print('The average VaR for the BL portfolio is ' 
      + str(np.mean([bl1_stress_test['VaR'], bl2_stress_test['VaR'], bl3_stress_test['VaR']])))
print('The average ES for the BL portfolio is ' 
      + str(np.mean([bl1_stress_test['ES'], bl2_stress_test['ES'], bl3_stress_test['ES']])))
print('\n')

# Black Litterman (Constrained Weights) Stress Testing on Validation Dataset
bl1_stress_test_constrained = stress_test(val_1, Optimal_Weights_BL['Optimal Weights'], alpha=0.9)
bl2_stress_test_constrained = stress_test(val_2, Optimal_Weights_BL['Optimal Weights'], alpha=0.9)
bl3_stress_test_constrained = stress_test(val_3, Optimal_Weights_BL['Optimal Weights'], alpha=0.9)

print('The average VaR for the BL constrained portfolio is ' 
      + str(np.mean([bl1_stress_test_constrained['VaR'], 
                     bl2_stress_test_constrained['VaR'], 
                     bl3_stress_test_constrained['VaR']])))
print('The average ES for the BL constrained portfolio is ' 
      + str(np.mean([bl1_stress_test_constrained['ES'], 
                     bl2_stress_test_constrained['ES'], 
                     bl3_stress_test_constrained['ES']])))


The average VaR for the market portfolio is 311000.0
The average ES for the market portfolio is 534460.3174603175


The average VaR for the ewp portfolio is 362895.8333333321
The average ES for the ewp portfolio is 614605.8201058186


The average VaR for the opt portfolio is 4473817.409509379
The average ES for the opt portfolio is 6028288.8234919645


The average VaR for the BL portfolio is 599574.598797597
The average ES for the BL portfolio is 855650.6956794189


The average VaR for the BL constrained portfolio is 323854.9792269934
The average ES for the BL constrained portfolio is 504972.0362922591
